In [1]:
# phase2_distress_detection_improved.py
# =========================================================
# Phase 2 – Distress Detection (Multi-label, CUDA-ready, improved for minority classes)
# =========================================================

import os
import json
import random
import numpy as np
import pandas as pd
import torch

from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from transformers import (
    RobertaTokenizer,
    RobertaConfig,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed,
)

# --------------------------
# 0) Reproducibility & Device
# --------------------------
SEED = 42
set_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except Exception:
        pass

# --------------------------
# 1) Paths & Config
# --------------------------
DATA_PATH = "../data/dataset1_conditions/processed/multilabled_clean.csv"
OUTPUT_DIR = "../results/models/distress_model"
LOG_DIR = "../results/logs"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

TEXT_COL = "cleaned_text"
LABEL_COLS = ["Normal", "Depression", "Suicidal", "Anxiety", "Bipolar"]
MODEL_NAME = "roberta-base"
MAX_LEN = 128
TEST_SIZE = 0.2

# --------------------------
# 2) Load data & quick checks
# --------------------------
df = pd.read_csv(DATA_PATH)
missing_cols = [c for c in [TEXT_COL] + LABEL_COLS if c not in df.columns]
if missing_cols:
    raise ValueError(f"Missing columns in CSV: {missing_cols}")

df[TEXT_COL] = df[TEXT_COL].astype(str)
for c in LABEL_COLS:
    df[c] = df[c].fillna(0).astype(int)

texts = df[TEXT_COL].tolist()
labels = df[LABEL_COLS].values  # (N,5)

print("Total samples:", len(texts))
print("Label counts:\n", df[LABEL_COLS].sum())

# --------------------------
# 3) Train / Val split
# --------------------------
X_train, X_val, y_train, y_val = train_test_split(
    texts, labels, test_size=TEST_SIZE, random_state=SEED
)
print(f"Train: {len(X_train)} | Val: {len(X_val)}")

# --------------------------
# 4) Tokenizer & encodings
# --------------------------
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

train_enc = tokenizer(X_train, truncation=True, padding=True, max_length=MAX_LEN)
val_enc   = tokenizer(X_val,   truncation=True, padding=True, max_length=MAX_LEN)

# --------------------------
# 5) PyTorch Dataset
# --------------------------
import torch.utils.data as tud

class DistressDataset(tud.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

train_ds = DistressDataset(train_enc, y_train)
val_ds   = DistressDataset(val_enc,   y_val)

# --------------------------
# 6) Sample weighting (oversample minority examples)
#    We compute per-sample weights based on inverse label frequency.
# --------------------------
label_counts = y_train.sum(axis=0)  # per-class counts on train
label_counts = np.maximum(label_counts, 1.0)  # avoid zeros

# class weight: inverse frequency (so rare classes get larger weight)
class_weight = (label_counts.max() / label_counts).astype(float)
print("Class weight (inverse freq):", dict(zip(LABEL_COLS, class_weight.tolist())))

# per-sample weight: sum of class_weight for the positive labels the sample has
sample_weights = (y_train * class_weight).sum(axis=1) + 1.0  # +1 to keep baseline
sample_weights = sample_weights.astype(float)

from torch.utils.data import WeightedRandomSampler
train_sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# --------------------------
# 7) pos_weight for BCEWithLogitsLoss (torch expects tensor)
#    pos_weight = (N - pos) / pos  (per-class)
# --------------------------
labels_tensor = torch.tensor(y_train, dtype=torch.float)
pos = labels_tensor.sum(dim=0)
neg = labels_tensor.shape[0] - pos
pos_ = torch.where(pos == 0, torch.ones_like(pos), pos)
pos_weight = (neg / pos_).to(device)
print("pos_weight:", pos_weight.tolist())

# --------------------------
# 8) Model setup
# --------------------------
num_labels = len(LABEL_COLS)
config = RobertaConfig.from_pretrained(MODEL_NAME, num_labels=num_labels, problem_type="multi_label_classification")
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, config=config)
model.to(device)

# --------------------------
# 9) Custom Trainer: focal + pos_weight + custom train dataloader (sampler)
# --------------------------
import torch.nn.functional as F
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader

class FocalBCETrainer(Trainer):
    def __init__(self, *args, train_sampler=None, pos_weight=None, focal_gamma=2.0, focal_alpha=1.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.train_sampler = train_sampler
        self.pos_weight = pos_weight
        self.focal_gamma = focal_gamma
        self.focal_alpha = focal_alpha

    def get_train_dataloader(self):
        if self.train_dataset is None:
            return None
        # make DataLoader that uses our sampler
        return DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
            sampler=self.train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers if hasattr(self.args, "dataloader_num_workers") else 0,
            pin_memory=self.args.dataloader_pin_memory,
        )

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        bce_loss = BCEWithLogitsLoss(reduction="none")
        loss = bce_loss(logits, labels.float())

        # Focal scaling
        probas = torch.sigmoid(logits)
        pt = torch.where(labels == 1, probas, 1 - probas)
        focal_factor = (1 - pt) ** self.focal_gamma
        loss = focal_factor * loss

        return (loss.mean(), outputs) if return_outputs else loss.mean()


# --------------------------
# 10) Metrics -- dynamic threshold tuning will be run after training.
# --------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs >= 0.5).astype(int)
    return {
        "micro_f1": f1_score(labels, preds, average="micro", zero_division=0),
        "macro_f1": f1_score(labels, preds, average="macro", zero_division=0),
        "accuracy": accuracy_score(labels, preds),
    }

# --------------------------
# 11) TrainingArguments (use fp16 on CUDA)
# --------------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,   # adjust to taste
    eval_strategy="steps",
    eval_steps=200,
    logging_steps=100,
    save_steps=200,
    save_total_limit=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    fp16=(device.type == "cuda"),
    dataloader_pin_memory=True,
    report_to="none",
    logging_dir=LOG_DIR,
)

# --------------------------
# 12) Create trainer and train
# --------------------------
trainer = FocalBCETrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_sampler=train_sampler,
    pos_weight=pos_weight,
    focal_gamma=2.0,
    focal_alpha=1.0,
)

train_result = trainer.train()
print("Train result:", train_result)

# --------------------------
# 13) Evaluate on validation and do threshold search (weighted macro-F1)
# --------------------------
pred_out = trainer.predict(val_ds)  # returns PredictionOutput
logits_val = pred_out.predictions  # (N_val, C)
labels_val = pred_out.label_ids

probs_val = 1 / (1 + np.exp(-logits_val))

# weighted macro-F1 search: give more weight to rare classes
train_label_counts = y_train.sum(axis=0).astype(float)
class_weights_for_thresh = (1.0 / (train_label_counts + 1e-9))
class_weights_for_thresh = class_weights_for_thresh / class_weights_for_thresh.sum()  # normalized
print("Threshold optimization class weights:", dict(zip(LABEL_COLS, class_weights_for_thresh.tolist())))

# grid to search
grid_vals = np.linspace(0.35, 0.85, 6)  # 6^5 = 7776 combos -> reasonable
best_score = -1.0
best_thresh = None

# For speed: generate all combinations and iterate
for combo in product(grid_vals, repeat=len(LABEL_COLS)):
    thresh = np.array(combo)
    preds = (probs_val >= thresh).astype(int)
    # if none predicted for sample, allow top-1 if top prob >= 0.3 (prevents empty set)
    none_mask = preds.sum(axis=1) == 0
    if none_mask.any():
        top_idx = probs_val[none_mask].argmax(axis=1)
        # set the top prediction to 1 when its prob >= 0.3
        top_probs = probs_val[none_mask, :].max(axis=1)
        pick_mask = top_probs >= 0.3
        rows_idx = np.where(none_mask)[0][pick_mask]
        cols_idx = probs_val[none_mask][:, :].argmax(axis=1)[pick_mask]
        preds[rows_idx, cols_idx] = 1

    # compute per-class f1
    f1s = np.array([f1_score(labels_val[:, i], preds[:, i], zero_division=0) for i in range(len(LABEL_COLS))])
    weighted_macro = float(np.sum(class_weights_for_thresh * f1s))
    if weighted_macro > best_score:
        best_score = weighted_macro
        best_thresh = thresh.copy()

best_thresholds = {LABEL_COLS[i]: float(best_thresh[i]) for i in range(len(LABEL_COLS))}
print("Selected global thresholds (weighted macro-F1):", best_thresholds)

# Optionally apply exclusivity: if any distress label flagged, remove 'Normal'
def apply_post_rules(preds_np):
    # preds_np: (n, C)
    preds_np = preds_np.copy()
    distress_idx = [i for i, lab in enumerate(LABEL_COLS) if lab != "Normal"]
    # if any distress predicted, set Normal=0
    any_distress = preds_np[:, distress_idx].sum(axis=1) > 0
    preds_np[any_distress, 0] = 0
    return preds_np

# Save thresholds
with open(os.path.join(OUTPUT_DIR, "thresholds.json"), "w") as f:
    json.dump(best_thresholds, f, indent=2)

# --------------------------
# 14) Save model + tokenizer (produces pytorch_model.bin & config.json)
# --------------------------
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved model and tokenizer to:", OUTPUT_DIR)

# --------------------------
# 15) Quick inference sanity checks (use saved thresholds & rules)
# --------------------------
model.eval()
sample_texts = [
    "I had a good day at work, everything went smoothly.",
    "I can’t get out of bed, I just want to sleep all day.",
    "Life is pointless and I don't want to continue.",
    "I swing between extreme highs and crushing lows.",
    "My heart races and I panic in crowded places.",
    "I feel mostly fine but worried before presentations.",
    "Sometimes I'm very energetic and talk too much, then crash.",
]

enc = tokenizer(sample_texts, truncation=True, padding=True, max_length=MAX_LEN, return_tensors="pt").to(device)
with torch.no_grad():
    logits = model(**enc).logits
    probs = torch.sigmoid(logits).cpu().numpy()

thresh_arr = np.array([best_thresholds[lab] for lab in LABEL_COLS])

for t, p in zip(sample_texts, probs):
    pred = (p >= thresh_arr).astype(int)
    # if none predicted, pick top-1 if prob>=0.3
    if pred.sum() == 0:
        top_i = int(p.argmax())
        if p[top_i] >= 0.3:
            pred[top_i] = 1
    pred = apply_post_rules(pred.reshape(1, -1))[0]
    print("\nText:", t)
    print("Probabilities:", np.round(p, 3))
    print("Predicted:", dict(zip(LABEL_COLS, pred.tolist())))

# --------------------------
# 16) Optionally save thresholds & a small calibration file
# --------------------------
# (we already saved thresholds.json)
if device.type == "cuda":
    torch.cuda.empty_cache()

print("Done.")


c:\Users\rijju\Desktop\mental-health-chatbot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
GPU: NVIDIA GeForce RTX 3050 Laptop GPU
Total samples: 53043
Label counts:
 Normal        16351
Depression    15404
Suicidal      10653
Anxiety        3888
Bipolar        2877
dtype: int64
Train: 42434 | Val: 10609
Class weight (inverse freq): {'Normal': 1.0, 'Depression': 1.05851755526658, 'Suicidal': 1.5082802547770702, 'Anxiety': 4.189128337085879, 'Bipolar': 5.670004353504571}
pos_weight: [2.25813889503479, 2.4487972259521484, 3.914186477661133, 12.648761749267578, 17.473661422729492]


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\rijju\AppData\Local\Temp\ipykernel_14596\1128289565.py:159: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalBCETrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Step,Training Loss,Validation Loss,Micro F1,Macro F1,Accuracy
200,0.069400,0.060420,0.719104,0.684786,0.661325
400,0.059200,0.053973,0.760513,0.746537,0.717881
600,0.053200,0.063857,0.714825,0.717519,0.677255
800,0.051600,0.050296,0.769867,0.753768,0.722028
1000,0.045400,0.049947,0.765058,0.761057,0.726836
1200,0.043600,0.050918,0.778223,0.764859,0.740126
1400,0.042500,0.051859,0.783892,0.764526,0.748610
1600,0.039700,0.044712,0.806920,0.798507,0.776322
1800,0.040800,0.048430,0.785237,0.781358,0.750683
2000,0.040400,0.046846,0.785140,0.776249,0.757470


Train result: TrainOutput(global_step=7959, training_loss=0.034278378008537995, metrics={'train_runtime': 4088.3262, 'train_samples_per_second': 31.138, 'train_steps_per_second': 1.947, 'total_flos': 8373866442988032.0, 'train_loss': 0.034278378008537995, 'epoch': 3.0})


Threshold optimization class weights: {'Normal': 0.07448273324169248, 'Depression': 0.0788412807005688, 'Suicidal': 0.1123408358702681, 'Anxiety': 0.31201772844630604, 'Bipolar': 0.4223174217411646}
Selected global thresholds (weighted macro-F1): {'Normal': 0.35, 'Depression': 0.35, 'Suicidal': 0.44999999999999996, 'Anxiety': 0.65, 'Bipolar': 0.75}
Saved model and tokenizer to: ../results/models/distress_model

Text: I had a good day at work, everything went smoothly.
Probabilities: [0.589 0.069 0.045 0.207 0.182]
Predicted: {'Normal': 1, 'Depression': 0, 'Suicidal': 0, 'Anxiety': 0, 'Bipolar': 0}

Text: I can’t get out of bed, I just want to sleep all day.
Probabilities: [0.099 0.459 0.292 0.11  0.068]
Predicted: {'Normal': 0, 'Depression': 1, 'Suicidal': 0, 'Anxiety': 0, 'Bipolar': 0}

Text: Life is pointless and I don't want to continue.
Probabilities: [0.057 0.499 0.499 0.064 0.036]
Predicted: {'Normal': 0, 'Depression': 1, 'Suicidal': 1, 'Anxiety': 0, 'Bipolar': 0}

Text: I swing 

In [6]:
import json
import os

# Order must match the one used during training!
conditions = ["Normal", "Depression", "Suicidal", "Anxiety", "Bipolar"]

idx2condition = {i: cond for i, cond in enumerate(conditions)}

output_path = "../results/models/distress_model/idx2condition.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    json.dump(idx2condition, f, indent=4)

print(f"Saved idx2condition.json at {output_path}")


Saved idx2condition.json at ../results/models/distress_model/idx2condition.json


In [4]:
sample_texts = [
    "I feel like I can’t handle life, and my chest is always tight with worry.",
    "I can’t sleep because my thoughts won’t stop, and everything feels meaningless.",
    "I feel so worthless, I don’t think I should keep going.",
    "Every day hurts, and sometimes I wish it would all just end.",
    "When I’m on a high, I talk too much, but then I crash and panic about everything.",

    "I swing between excitement and fear all the time.",
    "I feel mostly fine, but I get nervous before important meetings.",

"Life is okay, but I can’t stop worrying at night.",
"After my highs, I always fall into such a deep, dark place.",

"My mood swings take me from extreme energy to crushing sadness."

]
enc = tokenizer(sample_texts, truncation=True, padding=True, max_length=MAX_LEN, return_tensors="pt").to(device)
with torch.no_grad():
    logits = model(**enc).logits
    probs = torch.sigmoid(logits).cpu().numpy()

thresh_arr = np.array([best_thresholds[lab] for lab in LABEL_COLS])

for t, p in zip(sample_texts, probs):
    pred = (p >= thresh_arr).astype(int)
    # if none predicted, pick top-1 if prob>=0.3
    if pred.sum() == 0:
        top_i = int(p.argmax())
        if p[top_i] >= 0.3:
            pred[top_i] = 1
    pred = apply_post_rules(pred.reshape(1, -1))[0]
    print("\nText:", t)
    print("Probabilities:", np.round(p, 3))
    print("Predicted:", dict(zip(LABEL_COLS, pred.tolist())))



Text: I feel like I can’t handle life, and my chest is always tight with worry.
Probabilities: [0.064 0.207 0.087 0.561 0.054]
Predicted: {'Normal': 0, 'Depression': 0, 'Suicidal': 0, 'Anxiety': 1, 'Bipolar': 0}

Text: I can’t sleep because my thoughts won’t stop, and everything feels meaningless.
Probabilities: [0.149 0.517 0.249 0.172 0.086]
Predicted: {'Normal': 0, 'Depression': 1, 'Suicidal': 0, 'Anxiety': 0, 'Bipolar': 0}

Text: I feel so worthless, I don’t think I should keep going.
Probabilities: [0.079 0.468 0.494 0.06  0.038]
Predicted: {'Normal': 0, 'Depression': 1, 'Suicidal': 1, 'Anxiety': 0, 'Bipolar': 0}

Text: Every day hurts, and sometimes I wish it would all just end.
Probabilities: [0.055 0.432 0.51  0.06  0.04 ]
Predicted: {'Normal': 0, 'Depression': 1, 'Suicidal': 1, 'Anxiety': 0, 'Bipolar': 0}

Text: When I’m on a high, I talk too much, but then I crash and panic about everything.
Probabilities: [0.137 0.167 0.089 0.334 0.298]
Predicted: {'Normal': 0, 'Depression'